# Loading

In [1]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm

In [3]:
datasets = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/Chinese/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
        print(len(pieces))
        datasets.append(pieces)

17779
10054
11233


In [4]:
datasets[-1][-10]

{'content': '【离谱！土耳其男子帮搜救队找人 途中发现搜救对象是自己】据英国《每日邮报》9月29日报道，土耳其男子穆特鲁28日外出和朋友聚餐，醉酒后在林地中游荡。他的家人因为无法联系到他而报了警，警方随后出动救援队搜寻他的下落。在林地中游荡的穆特鲁正好碰到了正在搜寻他救援队，并热心地加入搜寻 ...全文',
 'label': 'real',
 'time': '2021-10-01 20:30:01'}

In [5]:
with open('../../dataset/Chinese/articles/articles.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

27505

In [6]:
random.sample(raw_articles, 1)[0]

{'_id': '5f0c71f1efe4e78f5b004de1',
 'time': 1580378821000,
 'time_format': '2020-01-30 18:07:01',
 'content_all': ['【辟谣 红绿灯将二十四小时改成红灯？',
  '假的！',
  '】今天，许多人的微信朋友圈和微信群出现了这样一条信息：通知，县各城区内，于今日凌晨24后所有路口红绿灯，统一24小时改红灯……',
  '安阳交警严正声明:未通过官方渠道发布过上述相关信息，该信息属不实信息，请勿相信。',
  '请大家自觉不造谣、不传谣、不信谣。',
  '疫情防控期间，请广大市民关注“安阳交警”微信微博抖音号，及时关注出行信息，另外，警方提醒:希望广大网民自觉抵制网络谣言，做到不信谣、不传谣，发现谣言及时举报，共同维护健康的网络环境和良好的社会秩序。',
  '对未经核实的信息，不要随意转发，更不要故意“杜撰改编”。',
  '对故意传播、散布虚假信息的，公安机关将依法严惩。',
  '来源：安阳交警']}

In [7]:
corpus = [''.join(a['content_all']) for a in raw_articles]
len(corpus)

27505

In [8]:
random.sample(corpus, 1)[0]

'#双黄连口服液可抑制新型冠状病毒##双黄莲蓉月饼 双黄连##护肤# 板蓝根和熏醋可以预防新冠病毒吗不可以板蓝根适用于治疗风热成冒等普通感冒的治疗。对冠状病毒是无效的;熏醋所含醋酸本身浓度很低，达不到消毒效果。而且，熏醋会对人的眼晴和呼吸道造成刺激。不建议这么做。丛博士护肤课堂'

In [9]:
model = bm25.init_model(corpus)

## Example

In [11]:
p = random.sample(random.sample(datasets, 1)[0], 1)[0]
print(p['content'], p['label'], p['time'])
print()

scores = bm25.get_a_query_scores(p['content'], model)
scores, scores.shape

for i, idx in enumerate(scores.argsort()[::-1][:10]):
    print('='*10, 'Top:',i, ', Score =', scores[idx], '='*10, '\n')
    print(raw_articles[idx]['time_format'])
    print(corpus[idx])
    print()

#安全与健康[超话]# 【海南省农业农村厅：成功研制出预防非洲猪瘟病毒注射剂】12日，海南省农业农村厅发布公告称，海南南药研究团队经对多种热带植物进行筛选，从中分离出预防非洲猪瘟病毒感染的复方成分，组方制成注射剂，用于预防非洲猪瘟病毒感染，经临床试验，显示出一定的预防效果。#预防非洲猪瘟病毒注射剂研制成功##安全与健康#  （南海网） fake 2019-06-12 00:00:00

========== Top: 0 , Score = 467.95243859892634 ========== 

2019-06-12 00:00:00
#海南视窗新闻眼# 【海南省农业农村厅：未发布猪瘟疫苗研制成功消息 媒体标题制作不准确】6月12日，有媒体报道称“海南成功研制出预防非洲猪瘟病毒注射剂”，引发网络关注。当天下午，海南省农业农村厅相关负责人告诉人民网记者，该厅并未发布所谓“成功研制出预防非洲猪瘟病毒注射剂”的公告，相关媒体在标题制作中不准确导致公众产生误解。据该负责人介绍，近日有相关企业发布研制“非洲猪瘟”药物一事，为了回应公众关注，海南省农业农村厅发布公告，称海南南药研究团队在省农业农村厅的支持下，对多种热带植物进行筛选，从中分离出预防非洲猪瘟病毒感染的复方成分，组方制成今珠多糖注射剂，用于预防非洲猪瘟病毒感染。经初步临床试验，显示出一定的预防效果。此后将对更多的样本、不同地区的感染程度、可能出现的病毒变异对预防作用的影响、预防维持时间、药物安全性、稳定性、性价比、原料资源情况及产业化的可行性等深入研究论证。有关研究进展情况，将由相关部门及时公布。“我们只是回应社会关注，为了防止相关企业或个人进行商业炒作，并没有具体点名，而且在发布公告的时候对媒体强调要原文刊发，没想到相关媒体在发布的时候修改了标题，导致公众误读。”该负责人表示，公告说的很清楚，“经初步临床试验，显示出一定的预防效果。”并没“研制成功”的字眼。网络资料显示，非洲猪瘟是一种猪的急性、烈性传染病，致死率最高可达100%。1921年，非洲猪瘟在肯尼亚被首次确诊，随后曾在多国肆虐，至今仍无有效疫苗。URL

========== Top: 1 , Score = 424.70931504315945 ========== 

2019-06-13 00:00:00
$海印股份 sz000861$ 【剧

## Total

In [1]:
for i, pieces in enumerate(datasets):
    queries = [p['content'] for p in pieces]
    scores_arr = bm25.get_total_scores(queries, corpus, bm25_model=model)
    np.save('data/Chinese/{}_{}.npy'.format(['train', 'val',
                                     'test'][i], scores_arr.shape), scores_arr)

# Export Top 10 (only articles published BEFORE the post）

## Loading

In [12]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm
import numpy as np
import pickle

In [2]:
datasets = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/Chinese/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
        print(len(pieces))
        datasets.append(pieces)

17779
10054
11233


In [3]:
datasets[-1][-10]

{'content': '【离谱！土耳其男子帮搜救队找人 途中发现搜救对象是自己】据英国《每日邮报》9月29日报道，土耳其男子穆特鲁28日外出和朋友聚餐，醉酒后在林地中游荡。他的家人因为无法联系到他而报了警，警方随后出动救援队搜寻他的下落。在林地中游荡的穆特鲁正好碰到了正在搜寻他救援队，并热心地加入搜寻 ...全文',
 'label': 'real',
 'time': '2021-10-01 20:30:01'}

In [4]:
with open('../../dataset/Chinese/articles/articles.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

27505

In [5]:
random.sample(raw_articles, 1)[0]

{'_id': '5f0c71f1efe4e78f5bffe5ab',
 'content': '#净网2018# #微博辟谣# 【网警辟谣：崇州捷普杀人？实为男子发病倒地】今日，成都网警在网络巡查中发现，百度贴吧内某网民称“崇州捷普杀人”，并配发一张微信聊天截图显示一男子倒在地上。成都网警随即向属地公安机关@平安崇州 大划派出所了解得知，此人是因为发病倒地头部受伤，人未死亡。成都网警在此提醒：断章取义亦是造谣手段，而真相只有一个。散布谣言，谎报险情、疫情、警情或者以其他方法故意扰乱公共秩序是违法行为，切勿尝试。@微博辟谣 @四川新闻网 @四川在线 @成都全搜索新闻网 @成都头条',
 'time': 1535472000000,
 'time_format': '2018-08-29 00:00:00',
 'content_all': ['#净网2018# #微博辟谣# 【网警辟谣：崇州捷普杀人？',
  '实为男子发病倒地】今日，成都网警在网络巡查中发现，百度贴吧内某网民称“崇州捷普杀人”，并配发一张微信聊天截图显示一男子倒在地上。',
  '成都网警随即向属地公安机关@平安崇州 大划派出所了解得知，此人是因为发病倒地头部受伤，人未死亡。',
  '成都网警在此提醒：断章取义亦是造谣手段，而真相只有一个。',
  '散布谣言，谎报险情、疫情、警情或者以其他方法故意扰乱公共秩序是违法行为，切勿尝试。',
  '@微博辟谣 @四川新闻网 @四川在线 @成都全搜索新闻网 @成都头条']}

## Find the nearest article

In [6]:
articles_times = [a['time_format'] for a in raw_articles]
len(articles_times)

27505

In [7]:
articles_times

['2004-08-19 16:59:00',
 '2005-08-12 12:48:00',
 '2007-02-02 00:00:00',
 '2007-02-02 00:00:00',
 '2007-02-02 17:02:59',
 '2007-02-02 17:02:59',
 '2007-02-03 05:15:00',
 '2007-02-03 05:15:00',
 '2007-02-03 14:44:00',
 '2007-02-03 14:44:00',
 '2007-02-03 16:00:00',
 '2007-02-03 16:00:00',
 '2008-03-29 16:10:00',
 '2008-05-25 10:18:39',
 '2008-12-24 08:42:00',
 '2009-03-05 11:03:00',
 '2009-06-08 14:48:00',
 '2009-09-09 00:00:00',
 '2009-09-16 15:36:13',
 '2009-10-15 00:00:00',
 '2009-12-24 16:15:00',
 '2010-01-03 08:08:00',
 '2010-04-13 08:16:00',
 '2010-05-24 00:00:00',
 '2010-05-24 17:53:00',
 '2010-05-25 05:30:00',
 '2010-05-26 00:00:00',
 '2010-05-26 05:58:00',
 '2010-06-01 00:00:00',
 '2010-06-21 00:00:00',
 '2010-07-08 22:09:00',
 '2010-07-09 08:51:00',
 '2010-07-27 15:33:55',
 '2010-07-30 00:00:00',
 '2010-09-06 00:00:00',
 '2010-09-06 00:00:00',
 '2010-09-06 00:00:00',
 '2010-10-29 01:15:00',
 '2010-11-11 00:00:00',
 '2010-11-20 00:00:00',
 '2010-11-25 00:00:00',
 '2010-11-26 00:

In [8]:
'2004-08-19 16:59:00' < '2005-08-12 12:48:00', '2004-08-19 16:59:00' <= '2004-08-19 17:00:00'

(True, True)

In [9]:
def find_the_nearest_article_idx(post_time):
    assert len(articles_times) == len(raw_articles)
    times = articles_times.copy()
    times.append(post_time)
    times.sort()
    return times.index(post_time) - 1

In [10]:
p = random.sample(random.sample(datasets, 1)[0], 1)[0]
print(p['content'], '\n', p['label'], p['time'])
print()

idx = find_the_nearest_article_idx(p['time'])

print(articles_times[idx])
print()
print(articles_times[idx-5:idx+5])

  青岛高速出行服务平台提示：接交警通知，因恶劣天气，龙青高速（龙青高速提供路况）暂时封闭。青新高速，前湾港2号疏港高速，沈海高速青岛段，青银高速青岛段（辽阳东路至机场口）正常开通。@青岛交通广播FM897 @青岛交通运输  
 real 2018-03-26 08:45:35

2018-03-26 07:55:46

['2018-03-26 00:00:00', '2018-03-26 00:00:00', '2018-03-26 00:00:00', '2018-03-26 00:00:00', '2018-03-26 00:00:00', '2018-03-26 07:55:46', '2018-03-26 08:57:48', '2018-03-26 13:40:00', '2018-03-26 14:46:00', '2018-03-26 22:49:44']


## Export

### Example

In [12]:
bm25_rank = np.load('./data/Chinese/bm25_train.npy')
bm25_rank.shape

(17779, 27505)

In [13]:
bm25_rank

array([[ 91.47818927, 101.77143036,  65.54788221, ...,  76.59504593,
         80.13756224, 103.18485174],
       [ 25.46381532,  47.09654764,  28.45159702, ...,  35.75664453,
         40.73598889,  35.14404523],
       [ 69.63658601, 102.58948743,  73.14568133, ...,  82.49750814,
         95.28780268,  88.0020962 ],
       ...,
       [ 66.7977584 ,  84.31078387,  62.16658365, ...,  96.54463128,
         92.4930803 ,  71.59008692],
       [119.76855949, 174.57958418, 134.86591663, ..., 137.8133134 ,
        163.91135925, 162.07462388],
       [ 51.76532487,  30.31205749,   4.06032093, ...,  13.65634979,
         14.60895622,  48.19212986]])

In [20]:
ranked_arr = (-bm25_rank).argsort()

In [21]:
ranked_arr

array([[12857, 20398,  9002, ..., 20852, 24336, 24390],
       [ 7944,  7773, 11154, ..., 25303, 22745,  7475],
       [ 8802, 11154, 11416, ...,  7478, 22022, 21997],
       ...,
       [17386, 13290,  9788, ...,   272, 15931,  8410],
       [19823, 17867, 17815, ...,  5714, 14781, 26214],
       [ 6117, 14254,  8115, ..., 17543, 17530,  7660]])

In [24]:
bm25_rank[0][20852],bm25_rank[0][24390],bm25_rank[0][20398],bm25_rank[0][12857],

(0.0, 0.0, 157.8619375905165, 432.86368847765397)

In [25]:
datasets[0][0]

{'content': '给孩子积德吧吉林省松原市深井子镇，养殖狐狸老板和老婆去打麻将 ，两个孩子饿了在家做饭 ，煤气罐爆炸 ，两个孩子都被炸死 现场哭得嘶声力竭，他们家后院堆积的都是他们活活扒皮的狐狸尸体。因果轮回早与晚种什么因得什么果缺什么别缺德 L美拍（通过 美拍 拍摄）',
 'label': 'fake',
 'time': '2017-12-13 13:22:00'}

In [26]:
raw_articles[12857]

{'_id': '5f1172acb9cce3edf314359e',
 'content': '<sred>给孩子积德吧吉林省松原市深井子镇，养殖狐狸老板和老婆去打麻将 ，两个孩子饿了在家做饭 ，煤气罐爆炸 ，两个孩子都被炸死 现场哭得嘶声力竭，他们家后院堆积的都是他们活活扒皮的狐狸尸体。因果轮回早与晚种什么因得什么果缺什么别缺德 L美拍（通过 美拍 拍摄） \u200b\u200b\u200b\u200b<ered><br>经查，该微博称“吉林省松原市深井子镇，养殖狐狸老板和老婆去打麻将 ，两个孩子饿了在家做饭 ，煤气罐爆炸 ，两个孩子都被炸死”不实，松原网警巡查执法已辟谣，详情：http://t.cn/RTHcXGB，被举报人构成“发布不实信息”。 现根据《微博举报投诉操作细则》（http://service.account.weibo.com/roles/guiding）第19条，对被举报人处理如下：扣除信用积分2分。上述处理在公布后60分钟内生效。',
 'time': 1542165341545,
 'time_format': '2018-11-14 11:15:41',
 'content_all': ['给孩子积德吧吉林省松原市深井子镇，养殖狐狸老板和老婆去打麻将 ，两个孩子饿了在家做饭 ，煤气罐爆炸 ，两个孩子都被炸死 现场哭得嘶声力竭，他们家后院堆积的都是他们活活扒皮的狐狸尸体。',
  '因果轮回早与晚种什么因得什么果缺什么别缺德 L美拍（通过 美拍 拍摄） \u200b\u200b\u200b\u200b',
  '经查，该微博称“吉林省松原市深井子镇，养殖狐狸老板和老婆去打麻将 ，两个孩子饿了在家做饭 ，煤气罐爆炸 ，两个孩子都被炸死”不实，松原网警巡查执法已辟谣，详情：URL，被举报人构成“发布不实信息”。',
  ' 现根据《微博举报投诉操作细则》（URL）第19条，对被举报人处理如下：扣除信用积分2分。',
  '上述处理在公布后60分钟内生效。']}

In [27]:
idx = find_the_nearest_article_idx(datasets[0][0]['time'])
idx

5299

In [28]:
ranked_arr[0]

array([12857, 20398,  9002, ..., 20852, 24336, 24390])

In [37]:
valid_idxs = np.where(ranked_arr[0] <= idx, ranked_arr[0], -1)
valid_idxs = [i for i in valid_idxs.tolist() if i != -1]
valid_idxs[:10]

[910, 53, 3686, 1154, 3811, 2682, 3731, 721, 2314, 2384]

In [40]:
raw_articles[3686]

{'_id': '5f2ea72f8d1bbb4d1141b3b8',
 'content': '近日有关女德讲师丁璇的讲座视频在网络上引起沸议，关于其讲座中德道、妇女权益、历史倒车和扯淡的问题，网上已有很多评论，丁香医生就从医学的角度，来说说她讲座中涉及的医学问题吧。\xa0 <br> 宫颈糜烂是个病？ <br> 丁香医生 & 较真鉴定：很多人一听到「糜烂」二字，脑子里已经对它打上了「严重疾病」的标签。至于像视频里这样的，更直接和生活糜烂「道德低下」挂钩了。然而宫颈糜烂不是疾病。所谓「宫颈糜烂」是由于雌激素的作用，使得宫颈管内的柱状上皮长到宫颈外口以外，长出去的部分被称为「宫颈柱状上皮异位」，看着有点丑，但这只是一种正常的生理变化。宫颈糜烂更不等于宫颈炎，对后者的确诊要查衣原体和病菌而不是看是否宫颈糜烂。反而对宫颈糜烂的过度治疗尤其是切除部分宫颈组织的治疗，会增加流产和早产的风险。\xa0 <br> 宫颈糜烂会导致宫颈癌？ <br> 丁香医生 & 较真鉴定：至于宫颈糜烂和宫颈癌的关系，更没有到「导致」的地步，现阶段看大多数宫颈癌的原因是高危型HPV反复感染，而非「宫颈糜烂」转化而来的。所以宫颈糜烂和宫颈癌的关系，有点像黑痣和黑色素瘤的关系，总不能说你脸上长个黑痣，就是有肿瘤吧。对于宫颈癌的防治更多的是要定期做筛查，而不是担心什么宫颈糜烂。 <br> 人真的会「精尽人亡」？ <br> 丁香医生 & 较真鉴定：精液里主要就两个部分，精子和精浆液。 精子是生孩子必须的「小蝌蚪」，也就是一个雄性的生殖细胞，而精浆液中 90% 都是水！此外还有少量的脂肪、蛋白质之类的营养物质，用于给精子提供能量的。既然精液由精子与精浆液组成，精浆液主要成分就是水，只要喝足够的水就不会缺；而精子会不断在睾丸中产生，太多了反而溢出，也就是「遗精」这个现象的来源。研究发现，每天每立方厘米的睾丸组织能产生 200 万个精子，也就是说每个男性的睾丸每天能够产生几千万至上亿个精子。这些精子即使不排出体外，最终也会老化、死亡，被人体处理掉。面对如此强大的「生产能力」，请问这得有多大的消耗量才能让人精尽人亡？ <br> 孕期性行为会产生胎毒？ <br> 丁香医生 & 较真鉴定：孕期不能进行性生活？虽然有研究证实孕期大部分男女性欲会下降，国内关于孕期性行为的研究也凤毛麟角，但是根据国外 30 多年的研究统计

### Total

In [13]:
for t in ['train', 'val', 'test']:
    with open('../../dataset/Chinese/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
    print('Pieces: ', len(pieces))
    
    bm25_rank = np.load('./data/Chinese/bm25_{}.npy'.format(t))
    print(t, bm25_rank.shape)
    
    print('\nSorting...')
    ranked_arr = (-bm25_rank).argsort()
    print('Done. ranked_arr: {}\n'.format(ranked_arr.shape))
    
    ranked_articles = []
    zero_num = 0
    for i, p in enumerate(tqdm(pieces)):
        idx = find_the_nearest_article_idx(p['time'])
        
        valid_idxs = np.where(ranked_arr[i] <= idx, ranked_arr[i], -1)
        valid_idxs = [aid for aid in valid_idxs.tolist() if aid != -1]
        if len(valid_idxs) == 0:
            zero_num += 1
        
        ranked_articles.append(valid_idxs)
        
    with open('./data/Chinese/ranked_articles_{}.pkl'.format(t), 'wb') as f:
        pickle.dump(ranked_articles, f)
        
    print('Dataset: {}, zero_num = {}\n{}\n'.format(t, zero_num, '='*15))

Pieces:  17779
train (17779, 27505)

Sorting...


  0%|          | 55/17779 [00:00<00:32, 548.49it/s]

Done. ranked_arr: (17779, 27505)



100%|██████████| 17779/17779 [00:29<00:00, 609.30it/s]


Dataset: train, zero_num = 0

Pieces:  10054
val (10054, 27505)

Sorting...


...

Dataset: val, zero_num = 0

Pieces:  11233
test (11233, 27505)

Sorting...


  0%|          | 42/11233 [00:00<00:27, 413.89it/s]

Done. ranked_arr: (11233, 27505)



100%|██████████| 11233/11233 [00:33<00:00, 339.74it/s]


Dataset: test, zero_num = 0



## Top 10

In [3]:
import pickle
import random
import numpy as np
from tqdm import tqdm

In [5]:
TOP=10

for t in ['train', 'val', 'test']:
    with open('./data/Chinese/ranked_articles_{}.pkl'.format(t), 'rb') as f:
        ranked_articles = pickle.load(f)
    print('{}: {}'.format(t, len(ranked_articles)))
    
    # <Padding> value = -1
    top_arr = np.zeros((len(ranked_articles), TOP)) - 1
    for i, ranks in enumerate(tqdm(ranked_articles)):
        sz = min(TOP, len(ranks))
        top_arr[i, :sz] = ranks[:sz]
        
    file = './data/Chinese/Top{}_articles_{}.npy'.format(TOP, t)
    np.save(file, top_arr)

100%|██████████| 17779/17779 [00:00<00:00, 363730.30it/s]

train: 17779



100%|██████████| 10054/10054 [00:00<00:00, 339868.57it/s]

val: 10054



100%|██████████| 11233/11233 [00:00<00:00, 315032.04it/s]

test: 11233


In [6]:
top_arr.shape

(11233, 10)

In [8]:
top_arr

array([[12772.,  9686.,  5666., ..., 14571.,  4714., 11026.],
       [14945.,  8877.,   458., ...,  8852.,  2752.,   470.],
       [14927., 14936., 14945., ...,  8826.,  8392.,  8852.],
       ...,
       [18923.,  8964.,   684., ...,  8893.,   653., 18930.],
       [25646., 25653., 24893., ..., 26735., 24939., 24970.],
       [ 8802., 10785.,  9327., ..., 18717.,   998.,  8931.]])

# Check

In [1]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm

In [2]:
datasets = []
top_articles = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/Chinese/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
    print(len(pieces))
    
    arr = np.load('./data/Chinese/Top10_articles_{}.npy'.format(t))
    arr = np.array(arr, dtype=int)
    print(arr.shape)
    
    datasets.append(pieces)
    top_articles.append(arr)

17779
(17779, 10)
10054
(10054, 10)
11233
(11233, 10)


In [3]:
with open('../../dataset/Chinese/articles/articles.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

27505

In [5]:
t = random.randint(0,2)
pieces = datasets[t]
arr = top_articles[t]

i = random.randint(0, len(pieces)-1)
p = pieces[i]
rank = arr[i]

print(p['content'], p['label'], p['time'])
print()

for i, idx in enumerate(rank):
    print('='*20, 'Top:',i, '='*20, '\n')
    print(raw_articles[idx]['time_format'])
    print(raw_articles[idx]['content_all'])
    print()

美国康涅狄格州法院最近审判了一对老年夫妻离婚案，两人闹离婚的原因相当奇葩：84岁的道森因为不想和80岁妻子多萝西多说话过，竟然装聋作哑62年…#男人别装# 大娘说，他跟老头结婚六十多年，一直合计他是个聋哑人，甚至为此学了两年手语，但还是没法和丈夫沟通，对此丈夫的解释是自个儿眼神不好。六十多年里，大爷以聋哑人的身份跟大娘抚养出了6个孩子13个孙辈，没有人怀疑过他的身份。而大娘之所以发现，是偶然在网上看到了大爷唱K的视频。大娘感觉自己受到了欺骗，坚持要离婚。但大爷说，自己不健谈，媳妇话又多，他要是不装聋哑人，俩人早离婚了，他装了这么多年，都是为了这个家！这个故事告诉我们，不要唱K。 fake 2019-03-22 00:00:00

==================== Top: 0 ==================== 

2018-07-20 00:00:00
['只要一个眼神就能治百病？', ' 揭底“催眠大师”的超能骗局', '脑瘫患者能够快速康复；植物人能复苏；聋哑人能讲话，这些连现代医学技术都很难破解的专业难题，有人却高调宣称自己行，只要一个眼神就能医治百病，功效神奇。', '前段时间江苏泰州就出现了这样一位所谓的大师。', '今年22岁的江苏泰州小伙小于从小患有脑瘫，多年来一直四处求医治疗，始终未见任何效果。', '前不久，小于在网上认识了一位叫邱瀚民的所谓女大师，她自称是全球眼神催眠的创始人，能够把宇宙能量瞬间植入人体医治百病，小于抱着试试看的态度加入了女大师的微信群。', '热心的邱瀚民很快给小于发来了很多现场治愈患者的小视频。', '受害人 于某：当时她给我看的那个视频里面，有个哑巴本来不能说话，后来能说话了，经过她的治疗。', '我就被这个视频打动了，当时相信她啊，因为（我感觉）她说的很真啊。', '在所谓女大师的治疗下植物人也能当场复苏。', '看到她提供的一个个现场治愈患者的真实案例，久病不愈的小于对此深信不疑。', '按照约定的时间，小于在微信上缴纳了6000块钱的治疗费后，只身前往上海进行所谓的眼神催眠治疗。', '受害人 于某：她把我带到（房间）里面去，里面有两个助手，然后她先是眼睛盯着我看，看了大概三十秒左右，就是说这个催眠的过程，后来她两个助手就把我抬到两个椅子中间，头在一个椅子上，脚在一个椅子上，她站在我肚子上面，大概两三秒